# Task 3: Heart Disease Prediction (Updated for Custom Dataset)

### 🎯 Objective
Predict whether a patient is at risk of heart disease using health-related features.
This version is tailored for a dataset where:
- `sex` is 'Male'/'Female'
- Booleans are stored as 'TRUE'/'FALSE'
- `cp`, `thal`, `slope` etc. are categorical
- Target column is `num` (we'll map it to binary)


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

%matplotlib inline

In [15]:
# Load dataset (make sure heart.csv is in the same folder)
df = pd.read_csv('heart_disease_uci.csv')
df.head()

id  age     sex    dataset               cp  trestbps   chol    fbs  \
0   1   63    Male  Cleveland   typical angina     145.0  233.0   True   
1   2   67    Male  Cleveland     asymptomatic     160.0  286.0  False   
2   3   67    Male  Cleveland     asymptomatic     120.0  229.0  False   
3   4   37    Male  Cleveland      non-anginal     130.0  250.0  False   
4   5   41  Female  Cleveland  atypical angina     130.0  204.0  False   

          restecg  thalch  exang  oldpeak        slope   ca  \
0  lv hypertrophy   150.0  False      2.3  downsloping  0.0   
1  lv hypertrophy   108.0   True      1.5         flat  3.0   
2  lv hypertrophy   129.0   True      2.6         flat  2.0   
3          normal   187.0  False      3.5  downsloping  0.0   
4  lv hypertrophy   172.0  False      1.4    upsloping  0.0   

                thal  num  
0       fixed defect    0  
1             normal    2  
2  reversable defect    1  
3             normal    0  
4             normal    0

In [20]:
# Drop unnecessary columns if present
cols_to_drop = ['id', 'dataset']
df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)
df = df.dropna()  # Remove rows with missing values if any

In [37]:
df = df.fillna(0)

In [38]:
print(df.dtypes[df.isnull().any()])

Series([], dtype: object)


In [39]:
# Convert boolean string columns to binary 1/0
bool_cols = ['fbs', 'exang']
for col in bool_cols:
    df[col] = df[col].map({'TRUE': 1, 'FALSE': 0})

In [40]:
# Convert 'TRUE'/'FALSE' to 1/0
# df.replace({'TRUE': 1, 'FALSE': 0}, inplace=True)

In [41]:
# Convert 'sex' to binary
df['sex'] = df['sex'].map({'Male': 1, 'Female': 0})

In [42]:
if 'num' in df.columns:
    df['target'] = df['num'].apply(lambda x: 1 if x > 0 else 0)
    df.drop(columns=['num'], inplace=True)

In [43]:
# Convert 'num' (0 = no disease, >=1 = disease) to binary
# df['target'] = df['num'].apply(lambda x: 1 if x > 0 else 0)
# df.drop(columns=['num'], inplace=True)

In [45]:
# Safely one-hot encode only existing categorical columns
for col in ['cp', 'restecg', 'slope', 'thal']:
    if col in df.columns:
        df = pd.get_dummies(df, columns=[col], drop_first=True)

In [48]:
print(df.columns.tolist())

['age', 'sex', 'trestbps', 'chol', 'fbs', 'thalch', 'exang', 'oldpeak', 'ca', 'target']


In [47]:
# One-hot encode categorical columns
categorical_cols = ['cp', 'restecg', 'slope', 'thal']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

KeyError: "None of [Index(['cp', 'restecg', 'slope', 'thal'], dtype='object')] are in the [columns]"

In [28]:
# Split features and target
X = df.drop('target', axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [14]:
# Logistic Regression
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)
y_pred_log = log_model.predict(X_test)

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# Decision Tree
tree_model = DecisionTreeClassifier()
tree_model.fit(X_train, y_train)
y_pred_tree = tree_model.predict(X_test)

In [ ]:
# Accuracy
print('Logistic Regression Accuracy:', accuracy_score(y_test, y_pred_log))
print('Decision Tree Accuracy:', accuracy_score(y_test, y_pred_tree))

In [ ]:
# Confusion Matrix
sns.heatmap(confusion_matrix(y_test, y_pred_log), annot=True, fmt='d', cmap='Blues')
plt.title('Logistic Regression - Confusion Matrix')
plt.show()

In [ ]:
# ROC Curve for Logistic Regression
y_prob_log = log_model.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_prob_log)
roc_auc = roc_auc_score(y_test, y_prob_log)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'Logistic Regression (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.grid(True)
plt.show()

### ✅ Conclusion
- Dataset was preprocessed: booleans converted, categorical encoded, and target binarized.
- Two models tested: Logistic Regression and Decision Tree.
- Evaluated with accuracy, confusion matrix, and ROC AUC.
- Ready for further documentation and submission.